In [ ]:
try:
    import google.colab
    !rm -rf /content/SportWarzSim
    !git clone https://github.com/daltontf/SportWarzSim.git 
    %cd SportWarzSim
except ImportError:
    pass

In [ ]:
from sportwarz import load_counties_geojson

counties_geojson = load_counties_geojson()

In [ ]:
from sportwarz import load_counties_data

co_data_frame = load_counties_data(counties_geojson)

In [ ]:
from sportwarz import load_leagues, Team, Coordinates

league_name = "MLB"

leagues = {
    league_name: {  },
}    

load_leagues(leagues)

leagues[league_name]["json"]["teams"] = [
    Team(
    name = "Metropolis Meteors" ,
    L = 1.0,
    S = 0.0,
    venue = "Metropolis Park",
    state = "Delaware",
    color = "green",
    coordinates = Coordinates(
        lat =  38.770688926039234, 
        lon = -75.19401893090962
)),
    Team(
    name = "Gotham City Knights" ,
    L = 1.5,
    S = 0.0,
    venue = "Gotham Park",
    state = "New Jersey",
    color = "navy",
    coordinates = Coordinates(
        lat =  39.090146894230486, 
        lon = -74.84258836712888
))
]

In [ ]:
from sportwarz import calculate_distances

calculate_distances(leagues, co_data_frame) 

In [ ]:
from sportwarz import compute_shares
        
compute_shares(leagues, co_data_frame)  

In [ ]:
from sportwarz import compute_output_dataframes

compute_output_dataframes(leagues, co_data_frame)       

In [ ]:
from sportwarz import render_map, reset_county_styles, heatmap_counties

reset_county_styles(counties_geojson)
heatmap_counties(leagues, counties_geojson, league_name) 

render_map(leagues, league_name, counties_geojson, co_data_frame)